In [1]:
from IPython.display import HTML
from subprocess import getoutput
s = getoutput('nvidia-smi')
if 'K80' in s:
    gpu = 'K80'
elif 'T4' in s:
    gpu = 'T4'
elif 'P100' in s:
    gpu = 'P100'
else:
    gpu='DONT PROCEED'
display(HTML(f"<h1>{gpu}</h1>"))

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
%cd /content/gdrive/MyDrive/darknet
!chmod +x ./darknet
!./darknet

/content/gdrive/MyDrive/darknet
usage: ./darknet <function>


In [4]:
import os
import random
folder="jeans"
category="jeans_material"
if category=="":
    inputpath="../"+folder
    opPath=folder+"_tiny_model"
else:
    inputpath="../"+folder+'/'+category
    opPath=folder+"_"+category+"_tiny_model"
if opPath not in os.listdir():
    !mkdir $opPath

In [5]:
d={}
for i in sorted(os.listdir(inputpath)):
    if '.' not in i:
        print(i, end=" ")
        temp=[]
        try:
            temp.extend([inputpath+'/'+i+"/men/"+k for k in os.listdir(inputpath+'/'+i+"/men/")])
        except:
            pass
        try:
            temp.extend([inputpath+'/'+i+"/women/"+k for k in os.listdir(inputpath+'/'+i+"/women/")])
        except:
            pass-
        temp=[k for k in temp if (k[:3]!="ls:")&("classes" not in k)]
        temp.sort()
        count=0
        for j in temp:
            if ".txt" in j:
                count+=1
        if count>10:
            d[i]=temp
            print(len(temp))

denim 867


In [6]:
train=[]
for i in d.keys():
    for j in range(len(d[i])-1):
        if d[i][j][:d[i][j].rfind('.')]==d[i][j+1][:d[i][j+1].rfind('.')]:
            train.append(d[i][j])
            j+=1

In [7]:
count=0
classes={}
for i in d.keys():
    classes[count]=i
    for j in d[i]:
        if ".txt" in j:
            f=open(j,'r+')
            s=[k.split(" ") for k in f.read().split('\n')[:-1]]
            s=[[str(count)]+k[1:] for k in s]+[['']]
            s="\n".join([" ".join(k) for k in s])
            f.seek(0)
            f.write(s)
            f.close()
    count+=1

In [8]:
f=open(inputpath+"/classes.txt",'w')
c=[]
for i in range(len(classes)):
    c.append(classes[i])
f.write("\n".join(c))
f.close()

In [9]:
!cat $inputpath/classes.txt

denim

In [10]:
classesPath=inputpath+"/classes.txt"
f=open(classesPath,"r")
names=f.read()
f.close()
f=open(inputpath+"/obj.names","w")
f.write(names)
f.close()

In [11]:
numClass=len(names.split("\n"))
objdata=["classes= "+str(numClass),
         "train = "+inputpath+"/train.txt",
         "valid = "+inputpath+"/valid.txt",
         "names = "+inputpath+"/obj.names",
         "backup = "+opPath+"/"]
f=open(inputpath+"/obj.data","w")
f.write("\n".join(objdata))
f.close()

In [12]:
train=[i for i in train if ".txt" not in i]
train=[i for i in train if ".eps" not in i]
train=[i for i in train if ".jfif" not in i]
train=[i for i in train if ".gif" not in i]
random.shuffle(train)
l=len(train)
valid=train[int(l*5/6):]
train=train[:int(l*5/6)]
f=open(inputpath+'/train.txt','w')
f.write("\n".join(train))
f.close()
f=open(inputpath+'/valid.txt','w')
f.write("\n".join(valid))
f.close()

In [13]:
numClass

1

In [14]:
if "yolov4-tiny-custom.cfg" not in os.listdir(inputpath):
    !cp cfg/yolov4-tiny-custom.cfg $inputpath/yolov4-tiny-custom.cfg
else:
    !rm $inputpath/yolov4-tiny-custom.cfg
    !cp cfg/yolov4-tiny-custom.cfg $inputpath/yolov4-tiny-custom.cfg

In [15]:
#change number of class and filters
comm='s/classes=80/classes='+str(numClass)+"/"
!sed -i $comm $inputpath/yolov4-tiny-custom.cfg
comm='s/filters=255/filters='+str((numClass+5)*3)+"/"
!sed -i $comm $inputpath/yolov4-tiny-custom.cfg
 
#change number of steps and max batches
max_batches=max(6000,(numClass)*2000)
comm='s/max_batches\ =\ 500200/max_batches\ =\ '+str(max_batches)+"/"
!sed -i $comm $inputpath/yolov4-tiny-custom.cfg
comm='s/steps=400000,450000/steps='+str(int(0.8*max_batches))+','+str(int(0.9*max_batches))+"/"
!sed -i $comm $inputpath/yolov4-tiny-custom.cfg
 
netSize=416  #can be multiple of 32 or 608(but makes training slow)
comm='s/width=480/width='+str(netSize)+"/"
!sed -i $comm $inputpath/yolov4-tiny-custom.cfg
comm='s/height=480/height='+str(netSize)+"/"
!sed -i $comm $inputpath/yolov4-tiny-custom.cfg
 
batch=64
subdiv=1
comm='s/batch=64/batch='+str(batch)+'/'
!sed -i $comm $inputpath/yolov4-tiny-custom.cfg
comm='s/subdivisions=1/subdivisions='+str(subdiv)+'/'
!sed -i $comm $inputpath/yolov4-tiny-custom.cfg

In [16]:
if "yolov4.conv.137" not in os.listdir():
    !wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137
if "yolov4-tiny.weights" not in os.listdir():
    !wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.weights
if "yolov4-tiny.conv.29" not in os.listdir():
    !wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.conv.29

In [17]:
!./darknet detector train $inputpath/obj.data $inputpath/yolov4-tiny-custom.cfg yolov4-tiny.conv.29 -dont_show -map

Streaming output truncated to the last 5000 lines.
 total_bbox = 465815, rewritten_bbox = 0.000644 % 

 (next mAP calculation at 5600 iterations) 

 Tensor Cores are used.
 Last accuracy mAP@0.5 = 97.24 %, best = 97.57 % 
 5512: 0.003331, 0.027133 avg loss, 0.000026 rate, 0.576630 seconds, 352768 images, 0.225730 hours left
Loaded: 0.780458 seconds - performance bottleneck on CPU or Disk HDD/SSD
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 30 Avg (IOU: 0.916254), count: 75, class_loss = 0.036435, iou_loss = 0.436344, total_loss = 0.472778 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 37 Avg (IOU: 0.825666), count: 3, class_loss = 0.018582, iou_loss = 0.124628, total_loss = 0.143209 
 total_bbox = 465893, rewritten_bbox = 0.000644 % 

 (next mAP calculation at 5600 iterations) 

 Tensor Cores are used.
 Last accuracy mAP@0.5 = 97.24 %, best = 97.57 % 
 5513: 0.027649, 0.027184 avg loss, 0.000026 rate, 0.562959 seconds, 352832 images, 0.225455 

In [18]:
!./darknet detector train $inputpath/obj.data $inputpath/yolov4-tiny-custom.cfg $opPath/yolov4-tiny-custom_last.weights -dont_show -map

 CUDA-version: 11000 (11020), cuDNN: 7.6.5, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 3.2.0
 Prepare additional network for mAP calculation...
 0 : compute_capability = 750, cudnn_half = 1, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 1, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 2    416 x 416 x   3 ->  208 x 208 x  32 0.075 BF
   1 conv     64       3 x 3/ 2    208 x 208 x  32 ->  104 x 104 x  64 0.399 BF
   2 conv     64       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x  64 0.797 BF
   3 route  2 		                       1/2 ->  104 x 104 x  32 
   4 conv     32       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  32 0.199 BF
   5 conv     32       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  32 0.199 BF
   6 route  5 4 	                           ->  104 x 104 x  64 
   7 conv     64       1 x 1/ 1    104 x 104 x  64 ->  104 x 104 x  64 0.089 BF
   8 route  2 7 	  

In [19]:
!./darknet detector map $inputpath/obj.data $inputpath/yolov4-tiny-custom.cfg $opPath/yolov4-tiny-custom_best.weights

 CUDA-version: 11000 (11020), cuDNN: 7.6.5, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 3.2.0
 0 : compute_capability = 750, cudnn_half = 1, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 1, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 2    416 x 416 x   3 ->  208 x 208 x  32 0.075 BF
   1 conv     64       3 x 3/ 2    208 x 208 x  32 ->  104 x 104 x  64 0.399 BF
   2 conv     64       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x  64 0.797 BF
   3 route  2 		                       1/2 ->  104 x 104 x  32 
   4 conv     32       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  32 0.199 BF
   5 conv     32       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  32 0.199 BF
   6 route  5 4 	                           ->  104 x 104 x  64 
   7 conv     64       1 x 1/ 1    104 x 104 x  64 ->  104 x 104 x  64 0.089 BF
   8 route  2 7 	                           ->  104 x 104 x 128 
   9 